## AutoGluon local mode 활용 예시

- AutoGluon object detection 예시 : [참고](https://auto.gluon.ai/stable/tutorials/multimodal/object_detection/quick_start/quick_start_coco.html)
- AutoGluon의 object detection은 내부적으로 [mmdetection](https://github.com/open-mmlab/mmdetection)을 사용합니다.
- 앞의 노트북에서 데이터를 다운로드 했다고 가정합니다.

### 설치 관련
- AutoGluon 설치 시 pip로 설치하면 제대로 안되는 경우가 있으며, 그런 경우 source에서 설치가 필요합니다. AutoGluon은 내부적으로 여러 패키지를 활용하기 때문에 설치에 시간이 좀 소요됩니다.

In [ ]:
# 일반적인 방법으로 설치
# !pip -q install autogluon

# Source 를 활용해서 직접 설치
!git clone https://github.com/autogluon/autogluon
!cd autogluon && ./full_install.sh

In [ ]:
!pip -q install openmim

In [ ]:
!mim install mmcv-full
!pip -q install mmdet

- 아래 명령어를 통해서 autogluon 버전을 확인할 수 있습니다.
- `import autogluon` 명령어로 정상 설치를 확인할 수 있습니다.

In [ ]:
!pip list | grep autogluon

In [ ]:
import autogluon

In [ ]:
import os
import time

In [ ]:
data_dir = os.path.join("data", "tiny_motorbike")
train_path = os.path.join(data_dir, "Annotations", "trainval_cocoformat.json")
test_path = os.path.join(data_dir, "Annotations", "test_cocoformat.json")

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
presets = "best_quality"

- 아래 import를 통해 `MultiModalPredictor`를 사용할 수 있습니다.
- 만일 `ImportError` 발생 시 kernel restart 를 하는 것이 좋습니다.

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
# Init predictor
import uuid

model_path = f"./autogluon-model-{uuid.uuid4().hex}"

predictor = MultiModalPredictor(
    problem_type="object_detection",
    sample_data_path=train_path,
    presets=presets,
    path=model_path,
)

- Predictor를 선언하고, 곧바로 학습을 진행할 수 있습니다.

In [ ]:
start = time.time()
predictor.fit(train_path)  # Fit
train_end = time.time()

In [ ]:
print("This finetuning takes %.2f seconds." % (train_end - start))

In [ ]:
eval_start = time.time()
predictor.evaluate(test_path)
eval_end = time.time() - eval_start

In [ ]:
print("The evaluation takes %.2f seconds." % (eval_end))

- 아래에서 결과를 확인할 수 있습니다.

In [ ]:
pred = predictor.predict(test_path, save_results=True)

In [ ]:
from autogluon.multimodal.utils import Visualizer
from PIL import Image
from IPython.display import display

conf_threshold = 0.5  # Specify a confidence threshold to filter out unwanted boxes

for i in range(10):
    
    image_result = pred.iloc[i]

    img_path = image_result.image  # Select an image to visualize

    visualizer = Visualizer(img_path)  # Initialize the Visualizer
    out = visualizer.draw_instance_predictions(image_result, conf_threshold=conf_threshold)  # Draw detections
    visualized = out.get_image()  # Get the visualized image

    img = Image.fromarray(visualized, 'RGB')
    display(img)